# Installing and Using Gymnasium

### Basic Installation:

Gymnasium is a standard API for reinforcement learning, and a diverse collection of reference environments.

It is maintained and updated by the Farama foundation. See https://gymnasium.farama.org/

To install it, run

In [1]:
# pip install gymnasium

This installation does not include dependencies for all families of environments (there's a massive number, and some can be problematic to install on certain systems). You can install these dependencies for one family like

In [2]:
# pip install "gymnasium[atari]"
# pip install "gymnasium[accept-rom-license]"

or, to install all dependencies, use

In [3]:
# pip install "gymnasium[all]"

### Box2d

Box2d allows you to create basic environments. To install, run
 

In [4]:
#pip install "gymnasium[box2d]"

For box2d, Microsoft Visual C++ 14.0 or greater is required. 

See: https://stackoverflow.com/questions/64261546/how-to-solve-error-microsoft-visual-c-14-0-or-greater-is-required-when-inst


### Examples
Let's look at a few environments that are already available in the gymnasium library. 

First, we need to import gymnasium.



In [2]:
import gymnasium as gym

Now, let's look at some examples of the built in environments in gymnasium.

A good explanation of the codes below is at https://towardsdatascience.com/creating-a-custom-gym-environment-for-jupyter-notebooks-e17024474617

In [6]:
# Example 1: open lunar lander

env = gym.make("LunarLander-v2", render_mode="human")
observation, info = env.reset()

for _ in range(5):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

In [3]:
#Example 2: CartPole

###########################################
#         Stage 1 - Initialization
###########################################

# create the cartpole environment
env = gym.make('CartPole-v1', render_mode="human")

# run for 10 episodes
for episode in range(5):

  # put the environment into its start state
  env.reset()

###########################################
#            Stage 2 - Execution
###########################################

  # run until the episode completes
  terminated = False
  while not terminated:

    # show the environment
    env.render()

    # choose a random action
    action = env.action_space.sample()

    # take the action and get the information from the environment
    observation, reward, terminated, truncated, info = env.step(action)


###########################################
#           Stage 3 - Termination
###########################################

# terminate the environment
env.close()

It is also possible to input actions directly:

In [10]:
# Example 3: play car racing
# use arrows to play. to exit press esc

%run C:\Users\yshirai\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\box2d\car_racing.py


action ['+0.00', '+0.00', '+0.00']
step 0 total_reward +7.39

action ['+0.00', '+0.00', '+0.00']
step 200 total_reward +58.55


## Training Environments

To be explored. See https://gymnasium.farama.org/tutorials/training_agents/

## Creating a Custom Gymnasium Environment

Next we show an example of how to construct a custom environment that implements the Gymnasium interface.

All that’s required is a class that inherits from the Gym environment and that adds the set of functions described above.

The example is taken again from https://towardsdatascience.com/creating-a-custom-gym-environment-for-jupyter-notebooks-e17024474617

In [5]:
import numpy as np
from gymnasium.spaces import Discrete,MultiDiscrete


class BabyRobotEnv_v1(gym.Env):

    def __init__(self, **kwargs):
        super().__init__()

        # dimensions of the grid
        self.width = kwargs.get('width',3)
        self.height = kwargs.get('height',3)

        # define the maximum x and y values
        self.max_x = self.width - 1
        self.max_y = self.height - 1

        # there are 5 possible actions: move N,E,S,W or stay in same state
        self.action_space = Discrete(5)

        # the observation will be the coordinates of Baby Robot
        self.observation_space = MultiDiscrete([self.width, self.height])

        # Baby Robot's position in the grid
        self.x = 0
        self.y = 0

    def step(self, action):
        obs = np.array([self.x,self.y])
        reward = -1
        done = True
        truncated = False
        info = {}
        return obs, reward, done, truncated, info

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        # reset Baby Robot's position in the grid
        self.x = 0
        self.y = 0
        info = {}
        return np.array([self.x,self.y]),info

    def render(self):
        pass


In [6]:
# create an instance of our custom environment
env = BabyRobotEnv_v1()

# use the Gymnasium 'check_env' function to check the environment
# - returns nothing if the environment is verified as ok
from gymnasium.utils.env_checker import check_env
check_env(env)

c:\Users\yshirai\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\utils\env_checker.py:317: UserWarning: WARN: Not able to test alternative render modes due to the environment not having a spec. Try instantialising the environment through gymnasium.make
  logger.warn(


In [7]:
print(f'Action Space: {env.action_space}')
print(f'Action Space Sample: {env.action_space.sample()}')

print(f'Observation Space: {env.observation_space}')
print(f'Observation Space Sample: {env.observation_space.sample()}')

Action Space: Discrete(5)
Action Space Sample: 2
Observation Space: MultiDiscrete([3 3])
Observation Space Sample: [0 0]


### For future: setup script file for own environment class

Ideally, one would like to have our custom environment registered as a proper Gymnasium environment, that can be created using ‘gymnasium.make’. In this case, one needs to create appropriate directories, which can be done through the setuptools library, as well as the appropriate .py files. Many other steps may be required for the ultimate product, such being able to simply install babyrobot environment using pip. Also, once this is done, what do we do to actually run it?

It is still not clear to me how to do all this.

For the babyrobot environment, one can download the full github codes at https://github.com/WhatIThinkAbout/BabyRobotGym. The setup.py file should install the directories, which is the first step.


### For Future: create graphical rendering

To get more than just a single vector as final output, one needs to construct graphics for the environment. This is done for the babyrobot env at https://towardsdatascience.com/creating-a-custom-gym-environment-for-jupyter-notebooks-e8718f36547b

I leave this for later.

# Box2D Environment

For our satellites project, the discrete and multidiscrete spaces of babyrobot will not be enough.

For instance, the satellites visual cone is a polygon in Box2d, and it would be consuming too much time replicating it in the multidiscrete class. Also it seems that collisions were already modelled there.

First we go through the basic hello box2d example from the documentation in https://box2d.org/documentation/index.html

In [8]:
import Box2D

### Step 1: create a world

Every Box2D program begins with the creation of a b2World object.

b2World is the physics hub that manages memory, objects, and simulation.

You can allocate the physics world on the stack, heap, or data section.

In [9]:
g = (0,-10) # direction and magnitude of movement of each dynamic body at each point in time

#world = Box2D.b2World(gravity=g, doSleep=True)
world = Box2D.b2World(gravity=g)

#checks:
print('Total body count = ',world.bodyCount)
print('Gravity vector = ',world.gravity)

Total body count =  0
Gravity vector =  b2Vec2(0,-10)


### Step 2: Create a ground (i.e. static) body with the shape of a box and no mass

Bodies are built using the following steps:
1. Define a body with position, damping, etc.
2. Use the world object to create the body
3. Define fixtures with a shape, friction, density, etc.
4. Create fixtures on the body

In [ ]:
# 1
p = (0.0,-9)
groundBodyDef = Box2D.b2BodyDef(position = p)

# 2
GroundBody = world.CreateBody(groundBodyDef)

# CreateBody creates a body in the world.
#         Takes a single b2BodyDef argument, or kwargs to pass to a temporary b2BodyDef.
#         world.CreateBody(position=(1,2), angle=1) 
#         is short for:
#         world.CreateBody(b2BodyDef(position=(1,2), angle=1))

#         If the definition (or kwargs) sets 'fixtures', they will be created on the 
#         newly created body. A single fixture is also accepted.

#         CreateBody(..., fixtures=[])

#         This is short for:
#             body = CreateBody(...)
#             for fixture in []:
#                 body.CreateFixture(fixture)

#          'shapes' and 'shapeFixture' are also accepted:
#          CreateBody(..., shapes=[], shapeFixture=b2FixtureDef())

#         This is short for:
#             body = CreateBody(...)
#             body.CreateFixturesFromShapes(shapes=[], shapeFixture=b2FixtureDef())

# 3
groundBox = Box2D.b2PolygonShape(box = (50,10))

# 4
GroundBody.CreateFixture(shape = groundBox)

In [15]:
#checks:
print('Total body count = ',world.bodyCount)
print('Gravity vector = ',world.gravity)

Total body count =  1
Gravity vector =  b2Vec2(0,-10)


### Step 3: Create a Dynamic Body

So now we have a ground body. We can use the same technique to create a dynamic body. The main difference, besides dimensions, is that we must establish the dynamic body's mass properties.

First we create the body using CreateBody. By default bodies are static, so we should set the b2BodyType at construction time to make the body dynamic.

In [16]:
# Create a dynamic body with position p
p = (0.0,4.0)
Body = world.CreateDynamicBody(position=p) #CreateDynamicBody does not seem to accept the intermediate step 1 used above with CreateBody (where it optional)

# create its shape fixture
DynamicBox = Box2D.b2PolygonShape(box = (1,1))

# attach the shape fixture to the body, along with other fixtures
Body.CreateFixture(shape = DynamicBox, density = 1, friction = 0.3)

#checks:
print('Total body count = ',world.bodyCount)
print('Gravity vector = ',world.gravity)

Total body count =  2
Gravity vector =  b2Vec2(0,-10)


### Simulating the World
After setting up the static box and the dynamic one, we can let gravity do its job, and we should see the dynamic box fall into the static one. 



As physics laws are solved each time step, the 

In [17]:
timeStep = 1/60 #Time step is set at 60 Hertz (1/60 seconds)
velocityIterations = 6 #Max iterations to correctly infer velocity from physics laws at each time st
positionIterations = 2 #Max iterations to correctly infer position from physics laws at each time step

for i in range(0,600):
    world.Step(timeStep, velocityIterations, positionIterations)
    position = Body.position
    angle = Body.angle
    print("{:.2f}".format(position.x), "{:.2f}".format(position.y), "{:.2f}".format(angle))


0.00 4.00 0.00
0.00 3.99 0.00
0.00 3.98 0.00
0.00 3.97 0.00
0.00 3.96 0.00
0.00 3.94 0.00
0.00 3.92 0.00
0.00 3.90 0.00
0.00 3.87 0.00
0.00 3.85 0.00
0.00 3.82 0.00
0.00 3.78 0.00
0.00 3.75 0.00
0.00 3.71 0.00
0.00 3.67 0.00
0.00 3.62 0.00
0.00 3.57 0.00
0.00 3.52 0.00
0.00 3.47 0.00
0.00 3.42 0.00
0.00 3.36 0.00
0.00 3.30 0.00
0.00 3.23 0.00
0.00 3.17 0.00
0.00 3.10 0.00
0.00 3.02 0.00
0.00 2.95 0.00
0.00 2.87 0.00
0.00 2.79 0.00
0.00 2.71 0.00
0.00 2.62 0.00
0.00 2.53 0.00
0.00 2.44 0.00
0.00 2.35 0.00
0.00 2.25 0.00
0.00 2.15 0.00
0.00 2.05 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 0.00
0.00 2.01 

Note: the body stops falling at level 2, as that is where its lower edge collides with the upper edge of the ground body 

In fact, recall that the ground body is positioned at (0,-10), with upper vertices at (0,-10)+(50,10) and (0,-10)+(-50,10), and similarly for the dynamic body.